# Flowsheet initialization
## ROK model = M3

In [1]:
# simulation case specifications
model_code = 3
case_name = 'Bakken'

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

# Import idaes model serializer to store initialized model
from idaes.core.util import model_serializer as ms

from pyomo.environ import (Constraint,
                           ConstraintList,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Set,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value,
                           minimize)

from src.unit_initialization import create_flowsheet, \
                                    define_models, \
                                    define_arcs, \
                                    set_unit_model_variables, \
                                    initialize_flowsheet, \
                                    set_scaling_factors,\
                                    update_model_after_initialization,\
                                    vapor_only_to_vapor_liquid_reformulate,\
                                    H106_inlet_vapor_reformulate

## Define flowsheet

In [3]:
m = create_flowsheet(model_code)

In [4]:
M_catalyst = 1167.003367 # kg

## Define equipment and connections

In [5]:
# define unit models
define_models(m, catalyst_mass = M_catalyst)
#define connections
define_arcs(m)

## Define inlet compositions

In [6]:
import pandas as pd

inlet_df = pd.read_csv('NGL_compositions.csv')

inlet_composition_dict = {}

for col in inlet_df.columns:
    if col == case_name:
        for i,r in inlet_df.iterrows():
            if r[col] == 0.0:
                inlet_composition_dict[r['Species']] = 1e-6
            else:
                inlet_composition_dict[r['Species']] = round(r[col],4)

inlet_flow_rate = 481.3888889

dehydro_conv_dict = {'ethane':0.3566,
                     'propane':0.6632,
                     'nbutane':0.5188,
#                      'ibutane':0.5188,
                    }

## Define constraints and set-points for equipment

In [7]:
set_unit_model_variables(m, model_code=model_code, feed_flow_rate = inlet_flow_rate, 
                         feed_temp = 308.0, feed_pressure = 700000.0,
                         inlet_composition_dict = inlet_composition_dict,
                         dehydro_conv_dict = dehydro_conv_dict)

## Scale model components

In [8]:
if model_code == 2 or model_code == 3:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-2, inlet_composition_dict = inlet_composition_dict)
elif model_code == 4 or model_code == 5:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-3, inlet_composition_dict = inlet_composition_dict)
else:
    pass

## Sequence for initialization

In [9]:
from pyomo.network import SequentialDecomposition
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 5

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

for o in heuristic_tear_set:
    print(o.name)
print("Order of initialization")
for o in order:
    print(o[0].name)    
    
def function(unit):
    unit.initialize(outlvl=idaeslog.INFO_HIGH)  

fs.s01
Order of initialization
fs.H101
fs.R101
fs.H102
fs.S101
fs.H103
fs.R102
fs.H104
fs.T102
fs.H105
fs.F101
fs.H106
fs.F102
fs.T104
fs.C101
fs.T105
fs.M102
fs.S102
fs.C102
fs.M101


## Initialize Flowsheet

In [10]:
inlet_flow_guess = inlet_flow_rate
tear_guesses = {"flow_mol": inlet_flow_guess,
        "temperature": 308.0,
        "pressure": 700000.0,
        "mole_frac_comp": inlet_composition_dict,
        }

In [11]:
initialize_flowsheet(m,tear_guesses = tear_guesses)

In [14]:
# Fix H105 DoFs: outlet T
m.fs.H105.outlet.temperature.fix()

# Fix F101 DoF: pressure drop
m.fs.F101.deltaP.fix()

# Fix H106 DoFs: outlet T and P
m.fs.H106.outlet.pressure.fix()
# m.fs.H106.outlet.temperature.fix()

# Fix F102 DoF: pressure drop
m.fs.F102.deltaP.fix()

In [15]:
DOF_initial = degrees_of_freedom(m)
print("The final DOF of initialized flowsheet is {0}".format(DOF_initial))

The final DOF of initialized flowsheet is 4


In [16]:
ms.to_json(m, fname="./initialization_files/CISTAR_unit_initialization_{}_M{}.json.gz".format(case_name, model_code))

{'etime_load_file': 0.09332799911499023,
 'etime_read_dict': 0.2864246368408203,
 'etime_read_suffixes': 0.015623807907104492}

## Add post-initialization constraints

In [17]:
update_model_after_initialization(m)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)

Liquid phase flow is zero. Translator block fs.T102 liquid phase composition equations are being modified...

Degenerate constraints removed and liquid phase composition values set to 1e-8

Translator block fs.T102 liquid phase composition equations already modified, no degenerate constraints remaining.



# Flowsheet convergence

## Unfix purge fraction for optimization

In [18]:
m.fs.C102.inlet.flow_mol.unfix()
m.fs.M101.recycle.flow_mol.unfix()

for k,v in inlet_composition_dict.items():
    m.fs.C102.inlet.mole_frac_comp[0, k].unfix()
    m.fs.M101.recycle.mole_frac_comp[0, k].unfix()
m.fs.S102.split_fraction[0, "purge"].unfix()
def recycle_converge(blk):
    return blk.M101.recycle.flow_mol[0] - blk.S102.inlet.flow_mol[0]*0.99 == 0.0

m.fs.recycle_converge_constraint = Constraint(rule=recycle_converge)

In [19]:
DOF_initial = degrees_of_freedom(m)
print("The final DOF of initialized flowsheet is {0}".format(DOF_initial))

In [20]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6,
                  'bound_push': 1e-8,
                  'max_iter':100
                 }
solve_status = solver.solve(m, tee=True)

Ipopt 3.13.2: tol=1e-06
bound_push=1e-08
max_iter=100


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale sci

In [21]:
# for i in large_residuals_set(m):
#     print(i,'\t',i())

## Fix purge fraction to 0.01 and re-solve

In [22]:
m.fs.S102.split_fraction[0, "purge"].unfix()
m.fs.S102.split_fraction[0, "purge"].fix(0.01)
m.fs.del_component(m.fs.recycle_converge_constraint)

In [23]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6,
                  'bound_push': 1e-8,
                  'max_iter':500
                 }
solve_status = solver.solve(m, tee=True)

Ipopt 3.13.2: tol=1e-06
bound_push=1e-08
max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale sci

In [24]:
ms.to_json(m, fname="./initialization_files/CISTAR_solve_constrained_{}_M{}_purge_{}.json.gz".format(case_name, model_code,round(m.fs.S102.split_fraction[0, "purge"](),3)))

In [25]:
m.fs.M101.report()
m.fs.H101.report()
m.fs.R101.report()
m.fs.H102.report()
m.fs.S101.report()
m.fs.E101.report()
m.fs.H103.report()
m.fs.R102.report()
m.fs.H104.report()
m.fs.T101.report()
m.fs.H105.report()
m.fs.F101.report()
m.fs.H106.report()
m.fs.F102.report()
m.fs.T102.report()
m.fs.T103.report()
m.fs.C101.report()
m.fs.M102.report()
m.fs.S102.report()
m.fs.C102.report()


Unit : fs.M101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                                     Units         feed      recycle    Outlet  
    Total Molar Flowrate          mole / second     481.39     2503.6     2984.9
    Total Mole Fraction hydrogen  dimensionless 1.0000e-06    0.10900   0.091424
    Total Mole Fraction methane   dimensionless   0.081100    0.16576    0.15211
    Total Mole Fraction ethane    dimensionless    0.46920    0.12809    0.18310
    Total Mole Fraction propane   dimensionless    0.30920   0.029006   0.074193
    Total Mole Fraction nbutane   dimensionless   0.078400   0.011807   0.022546
    Total Mole Fraction ibutane   dimensionless   0.026500   0.076237   0.068215
    Total Mole Fraction pentane   dimensionless   0.025800   0.015323   0.017012
    Total Mole Fraction hexane    dimensionless  0.0061000  0.0011751  0.0019694
  


Unit : fs.H105                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key       : Value       : Units : Fixed : Bounds
    Heat Duty : -5.5848e+06 :  watt : False : (None, 0.0)

------------------------------------------------------------------------------------
    Stream Table
                                           Units         Inlet     Outlet  
    Molar Flowrate ('Liq', 'ethane')    mole / second 1.0000e-08     3.5496
    Molar Flowrate ('Liq', 'propane')   mole / second 1.0000e-08     2.6061
    Molar Flowrate ('Liq', 'nbutane')   mole / second 1.0000e-08     3.4544
    Molar Flowrate ('Liq', 'ibutane')   mole / second 1.0000e-08     16.120
    Molar Flowrate ('Liq', 'pentane')   mole / second 1.0000e-08     13.849
    Molar Flowrate ('Liq', 'hexane')    mole / second 1.0000e-08     3.0979
    Molar Flowrate ('Liq', 'heptane')   mole


Unit : fs.F102                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value   : Units  : Fixed : Bounds
          Heat Duty :  0.0000 :   watt :  True : (None, None)
    Pressure Change : -81043. : pascal :  True : (None, 0.0)

------------------------------------------------------------------------------------
    Stream Table
                                                Units         Inlet    Vapor Outlet  Liquid Outlet
    Molar Flowrate ('Liq', 'ethane')         mole / second     1.6125            -             -  
    Molar Flowrate ('Liq', 'propane')        mole / second     1.8159            -             -  
    Molar Flowrate ('Liq', 'nbutane')        mole / second     2.8821            -             -  
    Molar Flowrate ('Liq', 'ibutane')        mole / second     12.987            -             -  
    Molar Flo